# Slitflow tutorial - Basic usage

This example provides an minimal overview of Slitflow by analyzing the trajectories of simulated random walks.

## 1. Installation
Current version of Slitflow is tested on Python >=3.8, <3.11. Please check your Python version by running the following cell.

In [ ]:
!python --version

Depending on your environment, additional setup may be needed to run this notebook.

Installing `ttf-mscorefonts-installer` is recommended for reproducing the figures.

In Google Colaboratory, execute the following cell to install it. By executing the following cell, you agree to the font license.

In [ ]:
!echo ttf-mscorefonts-installer msttcorefonts/accepted-mscorefonts-eula select true | debconf-set-selections \
  && apt-get install -y --no-install-recommends ttf-mscorefonts-installer
!rm -r /root/.cache/matplotlib/

<strong><em>!!! Restart the runtime now. !!!</strong></em>

Then install Slitflow as follows.

In [ ]:
! pip install slitflow

## 2. Preparing tutorial directory
In the latter part of the Basic tutorial (3.4), the results of each analysis
step are stored in the project directory. To create the project directory in
your workspace, follow these steps.

In [ ]:
import os

root_dir = "slitflow_tutorial"
project_dir = os.path.join(root_dir, "getting_started_basic")

# Create directories
if not os.path.isdir(root_dir):
    os.makedirs(root_dir)
if not os.path.isdir(project_dir):
    os.makedirs(project_dir)

## 3. Running the example
We usually import slitflow as follows:

In [ ]:
import slitflow as sf

### 3.1. Simulate random walks
Start by creating an Index object that defines the number of images and 
trajectories. Then we execute the ``run()`` method to make result data inside
the Index object.

See also [slitflow.tbl.create.Index](https://slitflow.readthedocs.io/en/latest/api/slitflow.tbl.create.html#slitflow.tbl.create.Index) for argument descriptions.

In [ ]:
D1 = sf.tbl.create.Index()
D1.run([], {"type": "trajectory", "index_counts": [2, 3], "split_depth": 0})
print(D1.data[0])

Then we append random walk coordinates to the index table by making the next 
analysis object.

See also [slitflow.trj.random.Walk2DCenter](https://slitflow.readthedocs.io/en/latest/api/slitflow.trj.random.html#slitflow.trj.random.Walk2DCenter) for argument descriptions.

In [ ]:
D2 = sf.trj.random.Walk2DCenter()
D2.run([D1], {"diff_coeff": 0.1, "interval": 0.1, "n_step": 5,
       "length_unit": "um", "seed": 1, "split_depth": 0})
print(D2.data[0])

### 3.2. Calculate the Mean Square Displacement
The following code calculates the MSD of each trajectory. Then MSDs are
averaged through all images.

See also [slitflow.trj.msd.Each](https://slitflow.readthedocs.io/en/latest/api/slitflow.trj.msd.html#slitflow.trj.msd.Each) and [slitflow.tbl.stat.Mean](https://slitflow.readthedocs.io/en/latest/api/slitflow.tbl.stat.html#slitflow.tbl.stat.Mean)

In [ ]:
D3 = sf.trj.msd.Each()
D3.run([D2], {"group_depth": 2, "split_depth": 0})

D4 = sf.tbl.stat.Mean()
D4.run([D3], {"calc_col": "msd", "index_cols": [
    "interval"], "split_depth": 0})
print(D4.data[0])

### 3.3. Make a figure image

<strong><em> By default, Slitflow disables figure display in a separate window via Matplotlib. To show figures in the Jupyter Notebook, import slitflow and run the following snippet.</em></strong>

In [ ]:
%matplotlib inline

Then plot the averaged MSD against the time interval. The graph style is
adjusted using style class and creates a figure tiff image.

See also [slitflow.fig.line.Simple](https://slitflow.readthedocs.io/en/latest/api/slitflow.fig.line.html#slitflow.fig.line.Simple), [slitflow.fig.style.Basic](https://slitflow.readthedocs.io/en/latest/api/slitflow.fig.style.html#slitflow.fig.style.Basic) and [slitflow.fig.figure.ToTiff](https://slitflow.readthedocs.io/en/latest/api/slitflow.fig.figure.html#slitflow.fig.figure.ToTiff)

In [ ]:
import matplotlib.pyplot as plt

D5 = sf.fig.line.Simple()
D5.run([D4], {"calc_cols": ["interval", "msd"],
              "err_col": "sem", "group_depth": 0, "split_depth": 0})

D6 = sf.fig.style.Basic()
D6.run([D5], {"limit": [-0.01, 0.52, -0.005, 0.205], "tick": [[0, 0.1, 0.2, 0.3, 0.4, 0.5], [0, 0.05,
                                                                                             0.1, 0.15, 0.2]], "label": ["Interval (s)", "MSD (\u03bcm$^{2}$)"], "format": ['%.1f', '%.2f']})

D7 = sf.fig.figure.ToTiff()
D7.run([D6], {"split_depth": 0})

plt.close()
plt.imshow(D7.to_imshow(0))
plt.axis("off")
plt.show()

If you encounter the `WARNING:matplotlib.font_manager:findfont: Font family 'Arial' not found.`, restart the runtime and begin from `2. Preparing tutorial directory`.

### 3.4. Run using a pipeline

The Pipeline class can perform all the above steps while saving data to a
project folder.

In [ ]:
PL = sf.manager.Pipeline(project_dir)
obs_names = ["Sample1"]
PL.add(sf.tbl.create.Index(), 0, (1, 1), 'channel1', 'index',
       obs_names, [], [],
       {"type": "trajectory", "index_counts": [2, 3], "split_depth": 0})
PL.add(sf.trj.random.Walk2DCenter(), 0, (1, 2), None, 'trj',
       obs_names, [(1, 1)], [0],
       {"diff_coeff": 0.1, "interval": 0.1, "n_step": 5, "length_unit": "um", "seed": 1, "split_depth": 0})
PL.add(sf.trj.msd.Each(), 0, (1, 3), None, 'msd',
       obs_names, [(1, 2)], [0],
       {"group_depth": 2, "split_depth": 0})
PL.add(sf.tbl.stat.Mean(), 0, (1, 4), None, 'avemsd',
       obs_names, [(1, 3)], [0],
       {"calc_col": "msd", "index_cols": ["interval"], "split_depth": 0})
PL.add(sf.fig.line.Simple(), 0, (1, 5), None, 'msd_fig',
       obs_names, [(1, 4)], [0],
       {"calc_cols": ["interval", "msd"], "err_col": "sem", "group_depth": 0, "split_depth": 0})
PL.add(sf.fig.style.Basic(), 0, (1, 6), None, 'msd_style',
       obs_names, [(1, 5)], [0],
       {"limit": [-0.01, 0.52, -0.005, 0.205], "tick": [[0, 0.1, 0.2, 0.3, 0.4, 0.5], [0, 0.05, 0.1, 0.15, 0.2]],
        "label": ["Interval (s)", "MSD (\u03bcm$^{2}$)"], "format": ['%.1f', '%.2f']})
PL.add(sf.fig.figure.ToTiff(), 0, (1, 7), None, 'msd_img',
       obs_names, [(1, 6)], [0],
       {"split_depth": 0})
PL.save("pipeline")
PL.run()

This code creates the following folder structure.
```python
slitflow_tutorial/getting_started_basic
    |--g0_config
    |    pipeline.csv
    |--g1_groupe1
        |--a1_index
        |    Sample1_index.csv
        |    Sample1_index.sf
        |    Sample1_index.sfx
        |--a2_trj
        |    Sample1_trj.csv
        |    Sample1_trj.sf
        |    Sample1_trj.sfx
        |--a3_msd
        |    Sample1_msd.csv
        |    Sample1_msd.sf
        |    Sample1_msd.sfx
        |--a4_avemsd
        |    Sample1_avemsd.csv
        |    Sample1_avemsd.sf
        |    Sample1_avemsd.sfx
        |--a5_msd_fig
        |    Sample1_msd_fig.fig
        |    Sample1_msd_fig.sf
        |    Sample1_msd_fig.sfx
        |--a6_msd_style
        |    Sample1_msd_style.fig
        |    Sample1_msd_style.sf
        |    Sample1_msd_style.sfx
        |--a7_msd_img
            Sample1_msd_img.tif
            Sample1_msd_img.sf
            Sample1_msd_img.sfx
```

We can use the make_flowchat() method of the pipeline object to create an
analytical flowchart diagram. The image is created as a PNG file in the
g0_config folder in the project directory.

In [ ]:
PL.make_flowchart("pipeline", "grp_ana")